#MoxField deck export to 11x17" proxy pages via ScryFall's API

In [ ]:
#@title 1. Enter your deck's name between the questions marks below (deck_name = "YOUR DECK NAME HERE"). Click "Show code" if necessary.
deck_name = "Commander - deck name"

In [1]:
#@title 2. Copy and paste MoxField Deck Export below. Make sure the quotations marks are on the same lines as your 1st and last card; no extra line break (text = """PASTE YOUR DECK HERE"""). Click "Show code" if necessary. Then, on the menubar, select Runtime > Run all and check your Google Drive for the results.
text = """1 Kambal, Profiteering Mayor (OTJ) 353
1 Access Tunnel (STX) 262
1 Adeline, Resplendent Cathar (J25) 57
1 Aerith Gainsborough (FIN) 4
1 Akroma's Memorial (FUT) 159
1 Anguished Unmaking (SOI) 242
1 Arcane Signet (P30M) 1F
1 Auron, Venerated Guardian (FIC) 10
1 Avacyn's Memorial (MIC) 69
1 Beregond of the Guard (LTC) 93
1 Bleachbone Verge (DFT) 250
1 Boromir, Warden of the Tower (LTR) 4
1 Brightclimb Pathway // Grimclimb Pathway (ZNR) 285
1 Cavern of Souls (MM3) 232
1 Caves of Koilos (C21) 283
1 Cecil, Dark Knight // Cecil, Redeemed Paladin (FIN) 380
1 Celestine, the Living Saint (40K) 10
1 Command Tower (CMR) 350
1 Commissar Severina Raine (40K) 112
1 Damnation (MM3) 63
1 Dawn's Truce (BLB) 9
1 Demonic Tutor (UMA) 93
1 Denethor, Ruling Steward (LTR) 198
1 Dion, Bahamut's Dominant // Bahamut, Warden of Light (FIN) 16
1 Eiganjo Castle (CHK) 275
1 Elena, Turk Recruit (FIC) 18
1 Flowering of the White Tree (LTR) 15
1 General Kudro of Drannith (IKO) 335
1 General Leo Cristophe (FIC) 135
1 General's Enforcer (IKO) 188
1 Generous Gift (CMM) 26
1 Godless Shrine (SLD) 128
1 Greymond, Avacyn's Stalwart (SLX) 18
1 Herald's Horn (C17) 53
1 Heroes' Podium (DMC) 185
1 Homeward Path (C13) 295
1 Horn of Gondor (LTR) 240
1 Inkshield (DSC) 221
1 Isolated Chapel (DOM) 241
1 Jadar, Ghoulcaller of Nephalia (MID) 108
1 Jerren, Corrupted Bishop // Ormendahl, the Corrupter (MID) 109
1 Kambal, Consul of Allocation (KLD) 183
1 Kindred Dominance (CMM) 640
1 Loran of the Third Path (BLC) 143
1 Loyal Retainers (CMM) 39
1 MacCready, Lamplight Mayor (PIP) 108
1 Mangara, the Diplomat (CMM) 42
1 Marsh Flats (MM3) 239
1 Masako the Humorless (CHK) 33
1 Nils, Discipline Enforcer (C21) 20
1 Orzhov Signet (C21) 254
1 Path of Ancestry (PLG21) C3
1 Path to Exile (MAR) 4
1 Patriarch's Bidding (MH2) 275
1 Phyrexian Reclamation (PLST) C15-133
3 Plains (HOU) 185
3 Plains (AKH) 250
1 Plaza of Heroes (DMU) 252
1 Ravos, Soultender (C16) 39 *F*
1 Reconnaissance (ACR) 82
1 Reflecting Pool (CNS) 210
1 Relic of Legends (DMU) 236
1 Reliquary Tower (PLST) M19-254
1 Rogue's Passage (C20) 303
1 Rosa, Resolute White Mage (FIN) 555
1 Scrubland (VMA) 313
1 Secluded Courtyard (NEO) 275
1 Sephiroth, Fabled SOLDIER // Sephiroth, One-Winged Angel (FIN) 115
1 Shadow, Mysterious Assassin (FIC) 50
1 Shizo, Death's Storehouse (CHK) 283
1 Sol Ring (DRC) 57
1 Squall, SeeD Mercenary (FIN) 402
3 Swamp (HOU) 187
3 Swamp (AKH) 252
1 Syr Konrad, the Grim (DSC) 158
1 Tainted Field (C21) 320
1 Tainted Sigil (ARB) 83
1 Talisman of Hierarchy (MH1) 233
1 Teferi's Protection (STA) 11
1 Tegan Jovanka (WHO) 28
1 Temple of Silence (M21) 255
1 Teysa Karlov (SLD) 1235
1 Thalia, Heretic Cathar (INR) 44
1 Thancred Waters (FIC) 31
1 Thought Vessel (DSC) 256
1 Tymna the Weaver (FCA) 18
1 Unclaimed Territory (PLST) XLN-258
1 Unspeakable Symbol (SCG) 79
1 Vault of Champions (CMR) 360
1 Vault of the Archangel (CLB) 927
1 Venat, Heart of Hydaelyn // Hydaelyn, the Mothercrystal (FIN) 329
1 Zenos yae Galvus // Shinryu, Transcendent Rival (FIN) 127"""

In [8]:
#@title Import Python libraries and set options

# import
import json
import pandas as pd
import requests
import shutil

from pathlib import Path
from google.colab import drive
from PIL import Image, ImageOps

# options
pd.options.mode.copy_on_write = True  # preparing for pandas 3.0 where everything is CoW

In [80]:
#@title Mount Google Drive and create directories if they don't exist
drive.mount('/gdrive')

# set Google drive directories
MTG_DIR = '/gdrive/MyDrive/mtg/'
DECKS_DIR = MTG_DIR + 'decks/'

# create Google drive directories if they don't exist
dirs = [MTG_DIR, DECKS_DIR]
for dir in dirs:
  Path(dir).mkdir(parents=True, exist_ok=True)
  print(f'{dir} exists: {Path(dir).exists()}')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/mtg/ exists: True
/gdrive/MyDrive/mtg/decks/ exists: True


In [127]:
#@title Functions

def get_moxfield_dataframe(text):
    # split text into lines
    cards = []
    for line in text.split('\n'):
        card_quantity, line = line.split(maxsplit=1)
        card_name, line = line.split(" (", maxsplit=1)
        card_set, line = line.split(")", maxsplit=1)

        # Moxfield adds ' *F*' to delineate a foiled card, which we don't want
        if '*' in line:
            card_number, line = line.split(" *", maxsplit=1)
            card_number = card_number.strip()
        else:
            card_number = line.strip()
        card = [card_quantity, card_name, card_set, card_number]
        cards.append(card)

    # set column names
    columns = ['quantity', 'name', 'mtg_set', 'collector_number']

    # create DataFrame
    df = pd.DataFrame(cards, columns=columns)

    # set columns to appropriate data types
    df['quantity'] = df['quantity'].astype(int)
    df['name'] = df['name'].astype(str)
    df['mtg_set'] = df['mtg_set'].astype(str).str.lower()  # set code to lowercase to match Scryfall data
    df['collector_number'] = df['collector_number'].astype(str)

    return df

def get_card_data_from_scryfall(set_code, collector_number):
    url = f'https://api.scryfall.com/cards/{set_code}/{collector_number}'
    response = requests.get(url)
    response.raise_for_status()
    card_data = response.json()

    return card_data

def get_download_dictionary_from_scryfall(df):
    total_cards_to_download = 0
    total_double_faced_cards = 0

    download_dict = {}

    for card in df.itertuples(name='Card'):
        set_code = card.mtg_set
        collector_number = card.collector_number

        for i in range(card.quantity):

            # convert i into zero-filled string to use as part of filename
            i += 1  # we'll use this as part of our filename so start with 1
            i = str(i).zfill(len(str(df['quantity'].max())))

            # get data from scryfall (in json form)
            scryfall_card_data = get_card_data_from_scryfall(card.mtg_set, card.collector_number)

            # double-faced cards don't have images located here so we try here
            try:
                image_url = scryfall_card_data['image_uris']['png']
                total_cards_to_download += 1

                # replace double backslashes in card.name with double-dash
                # so we don't try to add directories when saving images

                # NOTE: this applies to cards with 1 face, but multiple
                # names like Adventure cards
                if '//' in card.name:
                    card_name = card.name.replace('//', '--')
                else:
                    card_name = card.name

                # set image_output_file_name
                image_output_file_name = f'{card_name}_{card.mtg_set}-{card.collector_number}_{i}.png'

                # add to download_dict
                download_dict[image_output_file_name] = image_url

            except KeyError:
                total_double_faced_cards += 1
                total_cards_to_download += 2

                for card_face in scryfall_card_data['card_faces']:
                    card_name = card_face['name']
                    image_url = card_face['image_uris']['png']

                    # set image_output_file_name
                    image_output_file_name = f'{card_name}_{card.mtg_set}-{card.collector_number}_{i}.png'

                    # add to download_dict
                    download_dict[image_output_file_name] = image_url

    print(f'Total cards to download: {total_cards_to_download}')
    if total_double_faced_cards > 0:
        print(f'\tTotal double-faced cards: {total_double_faced_cards}\n')

    return download_dict

def download_file(file_url, output_file_path):
    # download file
    with open(str(output_file_path), 'wb') as out_file:
        shutil.copyfileobj(requests.get(file_url, stream = True).raw, out_file)

    if not output_file_path.exists():
        print(f'Failed to download {file_url} to {output_file_path}\n')
    else:
        print(f'Downloaded {file_url} to:\n\t{output_file_path}\n')

def is_image_valid(file_path):
    try:
        with Image.open(file_path) as image:
            image.verify()
            return True
    except (IOError, SyntaxError):
        return False

def download_images(deck_name, download_dict):

    # set image_output_dir_path on Google Drive and create it if it does not exist
    output_dir_path = Path(DECKS_DIR).joinpath(deck_name, '0. Scryfall images')
    output_dir_path.mkdir(parents=True, exist_ok=True)

    print(f'Downloading {len(download_dict)} images to:\n\t{output_dir_path} . . .')

    count = 1
    for image_name, image_url in download_dict.items():

        # set image_output_file_path
        output_file_path = output_dir_path.joinpath(image_name)
        if output_file_path.exists():
            if is_image_valid(output_file_path):
                # print(f'Image {count}\n\t{image_name} already exists in:\n\t{output_file_path}\n')
                continue  # skip to next iteration of loop
        else:
            # print(f'Downloading image {count}\n\t{image_name} to:\n\t{output_file_path} . . .\n')
            download_file(image_url, output_file_path)

    image_paths = sorted(output_dir_path.glob('*.png'))
    for image_path in image_paths:
        if not is_image_valid(image_path):
            print(f'!!! ERROR -- {image_path} is not valid! !!!')
            break
    else:
        print(f'\t{len(image_paths)} images downloaded\n')

    return output_dir_path

def process_images(deck_input_dir,
                   trim_pixels=12,
                   autocontrast_cutoff_percent=(5, 1)
                   ):
    input_dir_path = Path(deck_input_dir)
    output_dir_path = input_dir_path.parent.joinpath('1. Processed images')
    output_dir_path.mkdir(parents=True, exist_ok=True)
    original_png_paths = sorted(input_dir_path.glob('*.png'))
    print(f'Processing images in:\n\t{input_dir_path} . . .')

    for png_path in original_png_paths:

        # get card name in lowercase
        card_name = png_path.name.split('-', maxsplit=1)[0].lower()

        # open PIL image
        image = Image.open(png_path)

        # trim pixels off each side
        image = ImageOps.crop(image, trim_pixels)

        # autocontrast image with tone preservation
        if image.mode == 'RGBA':  # autocontrast will fail on (A)lpha channel
            # convert to RGB to remove alpha channel
            image = image.convert('RGB')

        image = ImageOps.autocontrast(image,
                                      cutoff=autocontrast_cutoff_percent,
                                      ignore=None,
                                      preserve_tone=True)
        output_name = png_path.stem + '.png'
        png_output_path = output_dir_path.joinpath(output_name)
        image.save(png_output_path)

    image_paths = sorted(output_dir_path.glob('*.png'))
    for image_path in image_paths:
        if not is_image_valid(image_path):
            print(f'!!! ERROR -- {image_path} is not valid! !!!')
            break
    else:
        print(f'\t{len(image_paths)} images processed')

    return output_dir_path

def get_total_sheets(total_cards, max_cards_per_sheet):
    if total_cards > max_cards_per_sheet:
        total_sheets = total_cards // max_cards_per_sheet
        if total_cards % max_cards_per_sheet > 0:
            total_sheets += 1
    else:
        total_sheets = 1
    return total_sheets

def layout_images(image_paths,
                  num_of_columns,
                  num_of_rows,
                  image_width_px,
                  image_height_px
                  ):
    image_size = (num_of_columns * image_width_px, num_of_rows * image_height_px)
    image_grid = Image.new('RGB', image_size, color='white')
    count = 0
    # Paste in each image
    for i_row in range(num_of_rows):
        for i_column in range(num_of_columns):
            left = i_column * image_width_px
            right = left + image_width_px
            upper = i_row * image_height_px
            lower = upper + image_height_px
            bbox = (left, upper, right, lower)
            try:
                image = Image.open(image_paths[count])
            except:
                break
            image_grid.paste(image, bbox)
            count += 1
    return image_grid

def make_proxy_pages(input_dir, ppi=300, paper_size='11x17'):
    if paper_size == '11x17':
        max_cards_per_sheet = 18
        paper_width_in = 17
        paper_height_in = 11
        max_columns = 6
        max_rows = 3
        paper_orientation = 'horizontal'
    elif paper_size == '8.5x11':
        max_cards_per_sheet = 12
        paper_width_in = 11
        paper_height_in = 8.5
        max_columns = 5
        max_rows = 2
        paper_orientation = 'horizontal'

    # get sorted list of processed PNG images
    image_paths = sorted(Path(input_dir).glob('*.png'))

    image = Image.open(image_paths[0])
    image_width_px, image_height_px = image.size

    # calculate number of sheets
    total_cards = len(image_paths)
    total_sheets = get_total_sheets(total_cards, max_cards_per_sheet)

    # get number of remainder cards for the last page
    remainder_cards = total_cards % max_cards_per_sheet

    # create and save proxy sheets
    for i_sheet in range(total_sheets):
        i_sheet += 1

        # set number_of_cards
        if total_sheets == 1:  # we have less than fits on 1 sheet
            number_of_cards = total_cards
        elif i_sheet == total_sheets and remainder_cards != 0:  # we have remainder cards on last sheet
            number_of_cards = remainder_cards
        else:  # we lucked out and printed a perfect number cards for our page size (it evernly divides)
            number_of_cards = max_cards_per_sheet

        print(f'i_sheet: {i_sheet}')
        print(f'\tnumber_of_cards: {number_of_cards}')

        # convert inches measurements to pixels
        paper_width_px = int(paper_width_in * ppi)
        paper_height_px = int(paper_height_in * ppi)

        print(f'\tExpected paper size:\n\t\t{paper_width_in} x {paper_height_in} @ {ppi} inches\n\t\t{paper_width_px} x {paper_height_px} pixels')

        print(f'Processing . . . Sheet {i_sheet}\n\tTotal cards: {number_of_cards}')

        # get images for grid from png_paths then remove them from list
        images_for_grid = image_paths[:number_of_cards]
        image_paths = image_paths[number_of_cards:]

        image_grid = layout_images(images_for_grid,
                                    max_columns,
                                    max_rows,
                                    image_width_px,
                                    image_height_px
                                    )

        grid_width, grid_height = image_grid.size

        # default orienation is horizontal with 36" wide roll paper
        if paper_orientation == 'horizontal':
            add_width_px = paper_width_px - grid_width
            add_height_px = paper_height_px - grid_height
        else:  # it's vertical
            add_width_px = paper_width_px - grid_height
            add_height_px = paper_height_px - grid_width

        border = (add_width_px // 2,
                    add_height_px // 2,
                    add_width_px // 2,
                    add_height_px // 2,
                    )

        image_grid = ImageOps.expand(image_grid.copy(), border=border, fill=(255, 255, 255))

        final_width_px, final_height_px = image_grid.size

        if final_width_px != paper_width_px:
            if final_width_px > paper_width_px:
                print(f'ERROR: final_width_px {final_width_px} > paper_width_px {paper_width_px}')
                continue  # on to next page loop
            elif final_width_px < paper_width_px:
                add_width_px = paper_width_px - final_width_px

                # if we're just 1 pixel off, then add it to the end of the document
                if add_width_px == 1:
                    border = (0, 0, add_width_px, 0)
                    image_grid = ImageOps.expand(image_grid.copy(), border=border, fill=(255, 255, 255))

        if final_height_px != paper_height_px:
            if final_height_px > paper_height_px:
                print(f'ERROR: final_height_px {final_height_px} > paper_height_px {paper_height_px}\n')
                continue  # on to next page loop
            elif final_height_px < paper_height_px:
                add_height_px = paper_height_px - final_height_px

                # if we're just 1 pixel off, then add it to the end of the document
                if add_height_px == 1:
                    border = (0, 0, 0, add_height_px)
                    image_grid = ImageOps.expand(image_grid.copy(), border=border, fill=(255, 255, 255))

        final_width_px, final_height_px = image_grid.size

        print(f'\tFinal paper size: {final_width_px} x {final_height_px} pixels\n')

        output_dir_path = Path(input_dir).parent.joinpath('2. Proxy Sheets')
        output_dir_path.mkdir(parents=True, exist_ok=True)
        proxy_sheet_name = f'{output_dir_path.parent.name}_{paper_orientation}_proxy-sheet_{str(i_sheet).zfill(len(str(total_sheets)))}.jpg'
        proxy_sheet_path = output_dir_path.joinpath(proxy_sheet_name)
        image_grid.save(proxy_sheet_path, dpi=(ppi, ppi), keeprgb=True)
        if proxy_sheet_path.exists():
            print(f'Saved proxy sheet to:\n\t{proxy_sheet_path}\n')

In [117]:
#@title Parse MoxField deck export into Pandas DataFrame, create download dictionary with links to scryfall, download then process the card images, create proxy pages for 11"x17" paper
df = get_moxfield_dataframe(text)
download_dict = get_download_dictionary_from_scryfall(df)
output_dir = download_images(deck_name, download_dict)
output_dir = process_images(output_dir)
make_proxy_pages(output_dir)

Total cards to download: 107
	Total double-faced cards: 7

	/gdrive/MyDrive/mtg/decks/commander - like humans do by SeraphSix/0. Scryfall images . . .
	107 images downloaded

Processing images in:
	/gdrive/MyDrive/mtg/decks/commander - like humans do by SeraphSix/0. Scryfall images . . .
	107 images processed
